In [1]:
import pandas as pd
import numpy as np
import csv
import re

In [2]:
test_file_plate_2_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Protein titration LissamineFA_22267.csv"
test_file_plate_2_2 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Protein titration LissamineFA_22741.csv"
test_file_list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listA.csv"
test_file_repeat_read_plate_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_plate1.csv"

In [ ]:
reader = csv.reader(open(test_file_repeat_read_plate_1, 'r'), delimiter=',') 
    
all_data = [] 
dfs = {}
counter = 0

# Create a list of lists with all data in the csv file
for row in reader:
    all_data.append(row)

# Find the total noumber of rows in the file
for index, item in enumerate(all_data):
    if item != [] and re.search(r"Exported with EnVision Workstation", item[0]) != None:   # Blank rows cannot be indexed 
        total_lines = index + 1  

# Find the data        
for index, item in enumerate(all_data):
    if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and all_data[index + 8] == []:
        skiprows = index + 9   # Set the skiprows parameter
        end_of_data = 0
        
        # find the index of the first blank row after the data table
        while end_of_data == 0:   
            for index, item in enumerate(all_data):
                if item == [] and index > skiprows:  
                    end_of_data = end_of_data + index
                    break
                    
        skipfooter = total_lines - end_of_data   # calucalte the skipfooter parameter
        
        # extract data into pandas df
        df = pd.read_csv(test_file_repeat_read_plate_1, sep=',', index_col=0, engine='python', skiprows=skiprows, skipfooter=skipfooter, encoding='utf-8')
        
        # add only the s and p channel df to dictionary 
        if counter != 0 and counter % 3 != 0:
            dfs[counter] = df
        
        counter = counter + 1

In [3]:
reader = csv.reader(open(test_file_repeat_read_plate_1, 'r'), delimiter=',') 
    
all_data = [] 
dfs = {}
counter = 0

# Create a list of lists with all data in the csv file
for row in reader:
    all_data.append(row)

for index, item in enumerate(all_data):
    if item != [] and re.search(r"Exported with EnVision Workstation", item[0]) != None:   # Blank rows cannot be indexed 
        total_lines = index + 1  
        
for index, item in enumerate(all_data):
    if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and all_data[index + 8] == []:
        skiprows = index + 9   
        end_of_data = 0
        
        while end_of_data == 0:   
            for index, item in enumerate(all_data):
                if item == [] and index > skiprows:  
                    end_of_data = end_of_data + index
                    break
                    
        skipfooter = total_lines - end_of_data
        
        df = pd.read_csv(test_file_repeat_read_plate_1, sep=',', index_col=0, engine='python', skiprows=skiprows, skipfooter=skipfooter, encoding='utf-8')
        
        if counter != 0 and counter % 3 != 0:
            dfs[counter] = df
        
        counter = counter + 1

341
9
25
skipfooter 316
0
35
51
skipfooter 290
1
61
77
skipfooter 264
2
87
103
skipfooter 238
3
113
129
skipfooter 212
4
139
155
skipfooter 186
5
